In [66]:
import pandas as pd
import zstandard as zstd
import os

In [67]:
data_dir = "trendingYT"
output_dir = "csv-extraction"
os.makedirs(output_dir, exist_ok=True)
csv_zst_files = [f for f in os.listdir(data_dir) if f.endswith(".csv.zst")]

for file_name in csv_zst_files:
    file_path = os.path.join(data_dir, file_name)
    base_name = os.path.splitext(file_name)[0]
    csv_path = os.path.join(output_dir, base_name)
    with open(file_path, 'rb') as compressed:
        dctx = zstd.ZstdDecompressor()
        with open(csv_path, 'wb') as out_csv:
            out_csv.write(dctx.decompress(compressed.read()))


In [68]:
# for files in output_dir:
df_ca = pd.read_csv(output_dir + "/CAvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_ca["country"] = "CA"
df_ca.head()

# df_us = pd.read_csv(output_dir + "/USvideos.csv")
# df_us["country"] = "US"
# df_us.head()


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA


In [69]:
df_ca.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40881 entries, 0 to 40880
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40881 non-null  object
 1   trending_date           40881 non-null  object
 2   title                   40881 non-null  object
 3   channel_title           40881 non-null  object
 4   category_id             40881 non-null  int64 
 5   publish_time            40881 non-null  object
 6   tags                    40881 non-null  object
 7   views                   40881 non-null  int64 
 8   likes                   40881 non-null  int64 
 9   dislikes                40881 non-null  int64 
 10  comment_count           40881 non-null  int64 
 11  thumbnail_link          40881 non-null  object
 12  comments_disabled       40881 non-null  bool  
 13  ratings_disabled        40881 non-null  bool  
 14  video_error_or_removed  40881 non-null  bool  
 15  de

In [70]:
df_ca.describe()

,category_id,views,likes,dislikes,comment_count
count,40881.000000,4.088100e+04,4.088100e+04,4.088100e+04,4.088100e+04
mean,20.795553,1.147036e+06,3.958269e+04,2.009195e+03,5.042975e+03
std,6.775054,3.390913e+06,1.326895e+05,1.900837e+04,2.157902e+04
min,1.000000,7.330000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,20.000000,1.439020e+05,2.191000e+03,9.900000e+01,4.170000e+02
50%,24.000000,3.712040e+05,8.780000e+03,3.030000e+02,1.301000e+03
75%,24.000000,9.633020e+05,2.871700e+04,9.500000e+02,3.713000e+03
max,43.000000,1.378431e+08,5.053338e+06,1.602383e+06,1.114800e+06


In [71]:
df_jp = pd.read_csv(output_dir + "/JPvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_jp["country"] = "JP"
df_jp.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,5ugKfHgsmYw,18.07.02,陸自ヘリ、垂直に落下＝路上の車が撮影,時事通信映像センター,25,2018-02-06T03:04:37.000Z,"事故|""佐賀""|""佐賀県""|""ヘリコプター""|""ヘリ""|""自衛隊""|""墜落""|""落下""|""現...",188085,591,189,0,https://i.ytimg.com/vi/5ugKfHgsmYw/default.jpg,True,False,False,佐賀県神埼市の民家に墜落した陸上自衛隊のＡＨ６４Ｄ戦闘ヘリコプターが垂直に落下する様子を、近...,JP
1,ohObafdd34Y,18.07.02,イッテQ お祭り男宮川×手越 巨大ブランコ②,神谷えりな Kamiya Erina 2,1,2018-02-06T04:01:56.000Z,[none],90929,442,88,174,https://i.ytimg.com/vi/ohObafdd34Y/default.jpg,False,False,False,NaN,JP
2,aBr2kKAHN6M,18.07.02,Live Views of Starman,SpaceX,28,2018-02-06T21:38:22.000Z,[none],6408303,165892,2331,3006,https://i.ytimg.com/vi/aBr2kKAHN6M/default.jpg,False,False,False,NaN,JP
3,5wNnwChvmsQ,18.07.02,東京ディズニーリゾートの元キャストが暴露した秘密5選,アシタノワダイ,25,2018-02-06T06:08:49.000Z,アシタノワダイ,96255,1165,277,545,https://i.ytimg.com/vi/5wNnwChvmsQ/default.jpg,False,False,False,東京ディズニーリゾートの元キャストが暴露した秘密5選\n\nかたまりクリエイトさま\n【検証...,JP
4,B7J47qFvdsk,18.07.02,榮倉奈々、衝撃の死んだふり！映画『家に帰ると妻が必ず死んだふりをしています。』特報,シネマトゥデイ,1,2018-02-06T02:30:00.000Z,[none],108408,1336,74,201,https://i.ytimg.com/vi/B7J47qFvdsk/default.jpg,False,False,False,家に帰ってきたサラリーマンのじゅん（安田顕）は、玄関で血を出して倒れている妻ちえ（榮倉奈々）...,JP


In [72]:
df_jp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20523 entries, 0 to 20522
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                20523 non-null  object
 1   trending_date           20523 non-null  object
 2   title                   20523 non-null  object
 3   channel_title           20523 non-null  object
 4   category_id             20523 non-null  int64 
 5   publish_time            20523 non-null  object
 6   tags                    20523 non-null  object
 7   views                   20523 non-null  int64 
 8   likes                   20523 non-null  int64 
 9   dislikes                20523 non-null  int64 
 10  comment_count           20523 non-null  int64 
 11  thumbnail_link          20523 non-null  object
 12  comments_disabled       20523 non-null  bool  
 13  ratings_disabled        20523 non-null  bool  
 14  video_error_or_removed  20523 non-null  bool  
 15  de

In [73]:
df_jp.loc[250:260]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
250,DHEgmES0HfU,18.08.02,テラスハウス オープニング ニュー ドアーズ 山チャンネル Vol.7「正直に言います、尊敬...,Netflix Japan,24,2018-02-06T10:00:05.000Z,"ネットフリックス|""ネトフリ""|""半田悠人""|""NETFLIX""|""テラスハウス""|""テラハ...",72459,575,25,250,https://i.ytimg.com/vi/DHEgmES0HfU/default.jpg,False,False,False,【山チャンネル】Netflix公式チャンネル独占公開！山里亮太（南海キャンディーズ）による『...,JP
251,0ovTFlKXcy0,18.08.02,[SUPERSTAR SMTOWN]東方神起プレイ動画,SUPERSTAR SMTOWN,24,2018-02-07T05:23:59.000Z,"東方神起|""SUPERSTAR SMTOWN""|""KPOP""|""音ゲー""|""リズムゲーム""|...",31013,1326,25,63,https://i.ytimg.com/vi/0ovTFlKXcy0/default.jpg,False,False,False,☆東方神起グッズがあたるキャンペーン開催中☆\n\n人気KPOPゲーム【SUPERSTAR ...,JP
252,7UQSgAjKYdk,18.08.02,【7th WEEK】 みずき、元恋人に「早く会いたい…」,TERRACE HOUSE / テラスハウス,24,2018-02-06T09:00:01.000Z,"TERRACE HOUSE|""テラスハウス""|""テラハ""|""映画""|""劇場版""|""NETFL...",114223,332,128,0,https://i.ytimg.com/vi/7UQSgAjKYdk/default.jpg,True,False,False,いよいよテラスハウスの新シーズンがNETFLIXにて配信スタート！ \nその未公開映像をYo...,JP
253,3Xn7nGqiJCU,18.08.02,Why Trump Doesn't Want To Talk To Robert Mueller,The Late Show with Stephen Colbert,24,2018-02-07T08:35:00.000Z,"The Late Show|""Stephen Colbert""|""Colbert""|""Lat...",957641,14094,267,1036,https://i.ytimg.com/vi/3Xn7nGqiJCU/default.jpg,False,False,False,With Trump still undecided about sitting down ...,JP
254,U7pzUOdzYYk,18.08.02,有賀さつき 病気が判明。独りで病魔と戦った決死の覚悟の理由に涙が止まらない…,刺さるニュース,22,2018-02-06T09:00:03.000Z,"刺さるニュース|""UCXYFibd4JtRzi-SsWCvcwww""|""有賀さつき""|""フジ...",223959,321,291,30,https://i.ytimg.com/vi/U7pzUOdzYYk/default.jpg,False,False,False,★チャンネル登録はこちらからお願いいたします★\n⇒https://www.youtube....,JP
255,Fmx-IEa83JU,18.08.02,2018-2-7-01【仮想通貨の投資方法は５つある】視聴者様から家庭用ビデオカメラが届きま...,【コンサルユーチューバー】「法令遵守」「セキュリティ」「ICO資金調達」経営コンサルタント・...,29,2018-02-07T07:49:10.000Z,[none],7757,123,26,24,https://i.ytimg.com/vi/Fmx-IEa83JU/default.jpg,False,False,False,★中野裕二へのご質問や仮想通貨取引所のパソコン設定（スマホ、タブレットも設定します）、人生相...,JP
256,U9Tkv9FjyNw,18.08.02,星野みなみ年表がおもしろすぎるwwww 乃木坂46,最新46と 48,22,2018-02-06T11:56:54.000Z,[none],156835,2555,37,493,https://i.ytimg.com/vi/U9Tkv9FjyNw/default.jpg,False,False,False,NaN,JP
257,NwkoKEOUYaA,18.08.02,猫の噛み癖が治ったと錯覚したあの夜,コロン BB,15,2018-02-07T07:00:01.000Z,"コロンBB|""ライス""|""猫""|""子猫""|""子供""|""面白""|""癒し""|""可愛い""|""ぬこ""...",12442,570,6,103,https://i.ytimg.com/vi/NwkoKEOUYaA/default.jpg,False,False,False,ハメられた,JP
258,T17xaPhX5dU,18.08.02,2/6!!1日千円!!インを頼りに今日も勝つ!!【競艇エブリディ029】,競艇エブリディ,24,2018-02-06T19:03:02.000Z,"競艇|""競艇予想""|""ギャンブル""|""ボートレース""|""競馬""|""競艇エブリディ""|""一日千...",10068,121,13,44,https://i.ytimg.com/vi/T17xaPhX5dU/default.jpg,False,False,False,【競艇エブリディ】\nその名の通り毎日配信予定\n\n少ない投資を心がけてますが\nヒートア...,JP
259,2X6O10gGfzQ,18.08.02,AND MESHのiPhoneX用ケースが新発売！やっと出た人気メッシュデザイン,Appleが大好きなんだよ,26,2018-02-07T08:30:24.000Z,"iPhone X|""AND MESH""|""アンドメッシュ""|""グラデーション""|""iPhon...",7198,112,17,72,https://i.ytimg.com/vi/2X6O10gGfzQ/default.jpg,False,False,False,ずっと今か今かと待っていた方もいらっしゃるのではないでしょうか。\nそのデザインでずっと人気...,JP


In [74]:
df_jp[ df_jp['video_id'] == 'DHEgmES0HfU' ]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
162,DHEgmES0HfU,18.08.02,テラスハウス オープニング ニュー ドアーズ 山チャンネル Vol.7「正直に言います、尊敬...,Netflix Japan,24,2018-02-06T10:00:05.000Z,"ネットフリックス|""ネトフリ""|""半田悠人""|""NETFLIX""|""テラスハウス""|""テラハ...",72459,575,25,250,https://i.ytimg.com/vi/DHEgmES0HfU/default.jpg,False,False,False,【山チャンネル】Netflix公式チャンネル独占公開！山里亮太（南海キャンディーズ）による『...,JP
206,DHEgmES0HfU,18.08.02,テラスハウス オープニング ニュー ドアーズ 山チャンネル Vol.7「正直に言います、尊敬...,Netflix Japan,24,2018-02-06T10:00:05.000Z,"ネットフリックス|""ネトフリ""|""半田悠人""|""NETFLIX""|""テラスハウス""|""テラハ...",72459,575,25,250,https://i.ytimg.com/vi/DHEgmES0HfU/default.jpg,False,False,False,【山チャンネル】Netflix公式チャンネル独占公開！山里亮太（南海キャンディーズ）による『...,JP
250,DHEgmES0HfU,18.08.02,テラスハウス オープニング ニュー ドアーズ 山チャンネル Vol.7「正直に言います、尊敬...,Netflix Japan,24,2018-02-06T10:00:05.000Z,"ネットフリックス|""ネトフリ""|""半田悠人""|""NETFLIX""|""テラスハウス""|""テラハ...",72459,575,25,250,https://i.ytimg.com/vi/DHEgmES0HfU/default.jpg,False,False,False,【山チャンネル】Netflix公式チャンネル独占公開！山里亮太（南海キャンディーズ）による『...,JP
294,DHEgmES0HfU,18.08.02,テラスハウス オープニング ニュー ドアーズ 山チャンネル Vol.7「正直に言います、尊敬...,Netflix Japan,24,2018-02-06T10:00:05.000Z,"ネットフリックス|""ネトフリ""|""半田悠人""|""NETFLIX""|""テラスハウス""|""テラハ...",72459,575,25,250,https://i.ytimg.com/vi/DHEgmES0HfU/default.jpg,False,False,False,【山チャンネル】Netflix公式チャンネル独占公開！山里亮太（南海キャンディーズ）による『...,JP


In [75]:
df_de = pd.read_csv(output_dir + "/DEvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_de["country"] = "DE"

df_fr = pd.read_csv(output_dir + "/FRvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_fr["country"] = "FR"

df_gb = pd.read_csv(output_dir + "/GBvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_gb["country"] = "GB"

df_in = pd.read_csv(output_dir + "/INvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_in["country"] = "IN"

df_jp = pd.read_csv(output_dir + "/JPvideos.csv", encoding = "utf-8", encoding_errors = "replace" )
df_jp["country"] = "JP"

df_kr = pd.read_csv(output_dir + "/KRvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_kr["country"] = "KR"

df_mx = pd.read_csv(output_dir + "/MXvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_mx["country"] = "MX"

df_ru = pd.read_csv(output_dir + "/RUvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_ru["country"] = "RU"

df_us = pd.read_csv(output_dir + "/USvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_us["country"] = "US"


In [76]:
from operator import concat


list_df = [df_ca, df_jp, df_de, df_fr, df_gb, df_in, df_kr, df_mx, df_ru, df_us]
col_names = list_df[0].columns
col_types = list_df[0].dtypes.to_dict()
df_result = pd.DataFrame({col: pd.Series(dtype=col_types[col]) for col in col_names})

for df in list_df:
    df_result = pd.concat([df_result, df])

df_result.head()
df_result.info()
df_result.describe()



<class 'pandas.core.frame.DataFrame'>
Index: 375942 entries, 0 to 40948
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   video_id                375942 non-null  object
 1   trending_date           375942 non-null  object
 2   title                   375942 non-null  object
 3   channel_title           375942 non-null  object
 4   category_id             375942 non-null  int64 
 5   publish_time            375942 non-null  object
 6   tags                    375942 non-null  object
 7   views                   375942 non-null  int64 
 8   likes                   375942 non-null  int64 
 9   dislikes                375942 non-null  int64 
 10  comment_count           375942 non-null  int64 
 11  thumbnail_link          375942 non-null  object
 12  comments_disabled       375942 non-null  bool  
 13  ratings_disabled        375942 non-null  bool  
 14  video_error_or_removed  375942 non-null  b

,category_id,views,likes,dislikes,comment_count
count,375942.000000,3.759420e+05,3.759420e+05,3.759420e+05,3.759420e+05
mean,20.232302,1.326568e+06,3.788431e+04,2.126107e+03,4.253775e+03
std,7.132413,7.098568e+06,1.654131e+05,2.248437e+04,2.545876e+04
min,1.000000,1.170000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,4.697800e+04,6.690000e+02,4.100000e+01,1.090000e+02
50%,23.000000,1.773705e+05,3.446000e+03,1.790000e+02,5.110000e+02
75%,24.000000,6.476792e+05,1.747650e+04,7.490000e+02,2.011000e+03
max,44.000000,4.245389e+08,5.613827e+06,1.944971e+06,1.626501e+06


In [77]:
for df in list_df:
    print(df.shape)


(40881, 17)
(20523, 17)
(40840, 17)
(40724, 17)
(38916, 17)
(37352, 17)
(34567, 17)
(40451, 17)
(40739, 17)
(40949, 17)


In [78]:
total_rows = sum(df.shape[0] for df in list_df)
print(total_rows)


375942


In [79]:
df_null_tag = df_result[df_result['tags'].isnull()]

# df_null_tag = df_result[df_result['tags'].str.len() <= 1]

df_null_tag.info()
df_null_tag.head()


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                0 non-null      object
 1   trending_date           0 non-null      object
 2   title                   0 non-null      object
 3   channel_title           0 non-null      object
 4   category_id             0 non-null      int64 
 5   publish_time            0 non-null      object
 6   tags                    0 non-null      object
 7   views                   0 non-null      int64 
 8   likes                   0 non-null      int64 
 9   dislikes                0 non-null      int64 
 10  comment_count           0 non-null      int64 
 11  thumbnail_link          0 non-null      object
 12  comments_disabled       0 non-null      bool  
 13  ratings_disabled        0 non-null      bool  
 14  video_error_or_removed  0 non-null      bool  
 15  description            

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country


In [80]:
total_views = df_result.groupby('channel_title')['views'].sum()
print(total_views)

total_views.to_csv('total_views_per_channel.csv')

channel_title
! 세상에 무슨일이                3942977
!!8時だよ面白ネタ大集合               50207
!BTS・TWICE まとめ               7310
!Los amorosos Virales¡       6069
!t Live                    240038
                           ...   
５分でできる DIY                 272607
ＢＡＳＨｔｖ                     116292
Ｋのフィッシングちゃんねる               37165
ＷＡＪＯＩＫＡ                    424129
💜Sandrea                   347155
Name: views, Length: 37824, dtype: int64


In [81]:
df_excluded = df_result[((df_result['comments_disabled'] == True) & (df_result['ratings_disabled'] == True)) | (df_result['video_error_or_removed'] == True)]
df_excluded.info()
df_excluded.head()

df_excluded.to_excel('excluded_videos.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 2620 entries, 173 to 34594
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                2620 non-null   object
 1   trending_date           2620 non-null   object
 2   title                   2620 non-null   object
 3   channel_title           2620 non-null   object
 4   category_id             2620 non-null   int64 
 5   publish_time            2620 non-null   object
 6   tags                    2620 non-null   object
 7   views                   2620 non-null   int64 
 8   likes                   2620 non-null   int64 
 9   dislikes                2620 non-null   int64 
 10  comment_count           2620 non-null   int64 
 11  thumbnail_link          2620 non-null   object
 12  comments_disabled       2620 non-null   bool  
 13  ratings_disabled        2620 non-null   bool  
 14  video_error_or_removed  2620 non-null   bool  
 15  descri

In [82]:
df_result[['comments_disabled', 'ratings_disabled', 'video_error_or_removed']].value_counts()

comments_disabled  ratings_disabled  video_error_or_removed
False              False             False                     362285
True               False             False                       6096
False              True              False                       4941
True               True              False                       2367
False              False             True                         253
Name: count, dtype: int64

In [83]:
df_no_excluded = df_result[~(((df_result['comments_disabled'] == True) & (df_result['ratings_disabled'] == True)) | (df_result['video_error_or_removed'] == True))]

df_no_excluded.info()
df_no_excluded.head()


<class 'pandas.core.frame.DataFrame'>
Index: 373322 entries, 0 to 40948
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   video_id                373322 non-null  object
 1   trending_date           373322 non-null  object
 2   title                   373322 non-null  object
 3   channel_title           373322 non-null  object
 4   category_id             373322 non-null  int64 
 5   publish_time            373322 non-null  object
 6   tags                    373322 non-null  object
 7   views                   373322 non-null  int64 
 8   likes                   373322 non-null  int64 
 9   dislikes                373322 non-null  int64 
 10  comment_count           373322 non-null  int64 
 11  thumbnail_link          373322 non-null  object
 12  comments_disabled       373322 non-null  bool  
 13  ratings_disabled        373322 non-null  bool  
 14  video_error_or_removed  373322 non-null  b

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA


In [84]:
df_no_excluded[['comments_disabled', 'ratings_disabled', 'video_error_or_removed']].value_counts()

comments_disabled  ratings_disabled  video_error_or_removed
False              False             False                     362285
True               False             False                       6096
False              True              False                       4941
Name: count, dtype: int64

In [85]:
for i in range(len(df_no_excluded)):
    if df_no_excluded['dislikes'][i] == 0:
        df_no_excluded['like_ratio'] = df_no_excluded['likes']
    else:
        df_no_excluded['like_ratio'] = df_no_excluded['likes'] / df_no_excluded['dislikes']

df_no_excluded.head()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df_no_excluded[df_no_excluded['dislikes'] == 0]
# it is strange that there are videos with 0 dislikes in trending youtube videos, probably botting problems

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
67,amEZKmJQ4Io,17.14.11,Drako - Watch Me Do It [Official Video],babygranderecords,10,2017-10-23T19:38:36.000Z,"Drako|""Watch Me Do It""|""watch me""|""migos""|""dap...",25887,0,0,6,https://i.ytimg.com/vi/amEZKmJQ4Io/default.jpg,False,True,False,PURCHASE / STREAM WATCH ME DO IT https://fanli...,CA
161,bsgocWaULIU,17.14.11,Troydon bent - Jam African,Troydon bent,10,2016-12-08T19:29:25.000Z,New Danchall music world reggae traditional so...,16921,187,0,5,https://i.ytimg.com/vi/bsgocWaULIU/default.jpg,False,False,False,Troydon bent performing Jam African produced b...,CA
235,wDEA3rpYHnI,17.15.11,Marie-Louise Arsenault réplique à Denise Bomba...,TV Classics,22,2017-11-13T01:26:37.000Z,Marie-Louise Arsenault qui réplique à Denise B...,15800,88,0,0,https://i.ytimg.com/vi/wDEA3rpYHnI/default.jpg,True,False,False,Moment favori à la télé québécoise: Marie-Loui...,CA
271,LfajgipdxmA,17.15.11,How does Yoonla work? Learning CPA,Anthonys Digital Lifestyle Dotcom,22,2017-11-14T10:57:38.000Z,"How does Yoonla work|""free work from home""|""le...",11810,1,0,0,https://i.ytimg.com/vi/LfajgipdxmA/default.jpg,False,False,False,How does Yoonla work? Learning CPA marketing t...,CA
313,8sdQ7dP9Drs,17.15.11,"Today's News: It's On - Get The Goods, Novembe...",Whistler Blackcomb,17,2017-11-14T18:18:21.000Z,[none],4851,31,0,0,https://i.ytimg.com/vi/8sdQ7dP9Drs/default.jpg,False,False,False,Today’s news: it’s on. Whistler Mountain will ...,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23446,8WbxbqwdWog,18.13.03,Camera Goes on Japanese Sushi Conveyor Belt Sh...,TkyoSam,22,2018-03-06T16:18:30.000Z,"東京サム|""jvlog""|""jvlogger""|""tkyosam""|""tokyo""|""sam...",639517,0,0,1059,https://i.ytimg.com/vi/8WbxbqwdWog/default.jpg,False,True,False,I will call this video A Camera's Journey Thro...,US
25551,zkrq7Kpd1so,18.24.03,ULTRA LIVE presents Ultra Music Festival 2018 ...,UMF TV,10,2018-03-24T07:55:21.000Z,"ultra music festival|""umf""|""mix""|""relive""|""aft...",1648886,0,0,548,https://i.ytimg.com/vi/zkrq7Kpd1so/default.jpg,False,True,False,Tune in here for the Ultra Music Festival 2018...,US
28550,9TUBf6l7FBg,18.14.04,Coachella 2018 LIVE Channel 1,Coachella,10,2018-04-05T06:48:28.000Z,[none],3598220,0,0,0,https://i.ytimg.com/vi/9TUBf6l7FBg/default_liv...,False,True,False,For more cameras and VR180 immersive experienc...,US
28751,9TUBf6l7FBg,18.15.04,Coachella 2018 LIVE Channel 1,Coachella,10,2018-04-05T06:48:28.000Z,[none],11137071,0,0,2,https://i.ytimg.com/vi/9TUBf6l7FBg/default_liv...,False,True,False,For more cameras and VR180 immersive experienc...,US
